In [1]:
import os
import numpy as np
from astropy.io import fits
from astropy.time import Time

from supernova import LightCurve2
import define_filters

In [2]:
def get_closest_photometry(date_obs, jd, phot_mag):
    date_indx = np.argmin(np.abs(jd - date_obs.jd))
    closest_mag = phot_mag[date_indx]
    date_sep = (jd - date_obs.jd)[date_indx]
    return closest_mag, date_sep

In [3]:
def convert_mag_to_flux(phot_mag, ifilter):
    bandpar_dict = define_filters.define_filters()
    flux = 10**(np.array(phot_mag)/-2.5)*bandpar_dict[ifilter][4]
    return flux

In [4]:
def get_cenwave(ifilter):
    bandpar_dict = define_filters.define_filters()
    cenwave = bandpar_dict[ifilter][2]
    return cenwave

In [5]:
def create_quba_input(fitsfile, ascii_file, ascii_dir, snobj):
    '''
    fitsfile : str
        name of the fits file that will be used to get the observation date from
    ascii_file: str
        name of the ascii spectrum file that will be used by iraf to calculate the scaled spectrum
    ascii_dir: str
        path to the ascii_file
    snobj: LightCurve2 object
        contains spectroscopy of your supernova
    '''
    date_obs = Time(fits.getval(fitsfile, 'date-obs', 0))
    spec_phase = (date_obs-Time(snobj.jdexpl, format='jd')).value
    quba_filename = os.path.join(ascii_dir,'quba_input_phot_day{}.ascii'.format(int(spec_phase)))
    quba_file = open(quba_filename, 'w')
    quba_hdr = '# name'
    quba_str = '{}'.format(ascii_file)
    quba_hdr = '{}{}'.format(quba_hdr, ' '*(len(quba_str)-len(quba_hdr)))
    for ifilter in snobj.jd.keys():
        cenwave = get_cenwave(ifilter)
        if cenwave > 3500:
            mag, date_sep = get_closest_photometry(date_obs, snobj.jd[ifilter], snobj.apparent_mag[ifilter])
            if (date_sep < 2) and (len(ifilter)==1):
                quba_str = '{}    {}'.format(quba_str, mag)
                quba_hdr = '{}    {}      '.format(quba_hdr, ifilter)
    quba_file.write('{}\n'.format(quba_hdr))
    quba_file.write(quba_str)
    quba_file.close()
    print('run from the commandline in iraf27 environment')
    print('\tcd {}'.format(ascii_dir) )
    print('\tqubascalespectra.py {} {}'.format(os.path.basename(quba_filename),
                                                '/Users/bostroem/Dropbox/DLT40_all/script/scalespectra/filters/'))

In [6]:
sn15oz = LightCurve2('asassn-15oz')
sn15oz.get_photometry()

In [7]:
EFOSC_DIR = '../data/spectra/EFOSC/'
create_quba_input(os.path.join(EFOSC_DIR, 'tASASSN-15oz_20160410_Gr13_Free_slit1.5_57723_1_e.fits'), 
                  'tASASSN_15oz_20160410_Gr13_Free_slit1.5_57723_1_e.asci', 
                  '/Users/bostroem/Desktop/research/asassn15oz/data/spectra/EFOSC/scaled', 
                 sn15oz)

run from the commandline in iraf27 environment
	cd /Users/bostroem/Desktop/research/asassn15oz/data/spectra/EFOSC/scaled
	qubascalespectra.py quba_input_phot_day227.ascii /Users/bostroem/Dropbox/DLT40_all/script/scalespectra/filters/


In [6]:
from astropy.io import fits
import os
fits.getval(os.path.join('/Users/bostroem/Desktop/research/asassn15oz/data/spectra/EFOSC','tASASSN-15oz_20160410_Gr13_Free_slit1.5_57723_1_e.fits'), 'date-obs', 0)

'2016-04-11T09:20:13.397'